In [ ]:
Step_no = rI = dA = 1

In [ ]:
import cv2, functools, gc, glob, math, numpy as np, os, pandas, random, tensorflow as tf, time, warnings
import keras
from keras import backend as K
from keras.applications.vgg16 import VGG16
from keras.callbacks import CSVLogger, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.layers import Activation, add, BatchNormalization, concatenate, Conv1D, Conv2D, Conv3D, Dense, dot, Dropout
from keras.layers import Flatten, GlobalAveragePooling2D, Input, Lambda, MaxPool1D, MaxPooling2D, multiply, Reshape
from keras.models import Model
from tensorflow.keras.optimizers import SGD,Adam
from PIL import Image, ImageEnhance
from skimage import io, transform
from sklearn import preprocessing
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
warnings.filterwarnings("ignore", message = "Numerical issues were encountered")
warnings.filterwarnings("ignore", message = "Creating an ndarray from ragged nested sequences")
warnings.filterwarnings("ignore", category = DeprecationWarning)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dir = "/content/drive/MyDrive/DATASET"
doc, k, l, lastLine, classnum = open("all_files.txt",'w'), 0, 0, "", -1
for root, dir, files in os.walk(dir):
    for file in files:
        k += 1
        if file.split('.')[-1]  in ['jpg', 'JPG']:
            l += 1
            print(os.path.join(root,file), file = doc)
print(k, l)
doc.close()
list_file = open("file_list.txt","w")
with open("all_files.txt") as f:
    line = f.readline()
    while line:
        thisline = line.split("/")[-3]+line.split("/")[-2]
        if lastLine != thisline:
            classnum, lastLine = classnum + 1, thisline
        list_file.write(line.split("\n")[0]+"\t"+str(classnum)+"\n")
        line = f.readline()
f.close()
list_file.close()

3302 3302


In [ ]:
def creat_list(path): # called inside main Function
    global Step_no
    print(str(Step_no) + ": creat_list Function called.")
    Step_no, lists = Step_no + 1, [[] for i in range(4)]
    with open(path) as f:
        line, k = f.readline(), 0
        while line:
            classnum = int(line.split("\t")[1]) # classnum: 1 for NE, 3 for EP, 2 for EH, 0 for EA
            lists_index = 0 if classnum == 1 else 1 if classnum == 3 else 2 if classnum == 2 else 3
            lists[lists_index].append(line.split("\t")[0])
            line, k = f.readline(), k + 1
    l = {"NE": len(lists[0]), "EP": len(lists[1]), "EH": len(lists[2]), "EA": len(lists[3])}
    print(k, l)
    f.close()
    return np.array(lists, dtype=object)

In [ ]:
def slice_train_test(array, i, K): # called inside cross_validation Function
    global Step_no
    print(str(Step_no) + ": slice_train_test Function called.")
    Step_no += 1
    exp_image_cnt_in_test = len(array) // K
    test_start = i * exp_image_cnt_in_test
    test_end = test_start + exp_image_cnt_in_test
    test_array = array[test_start : test_end]
    train_array = array[ : test_start] + array[test_end : ]
    return train_array, test_array # The output is the category-wise lists of train-data paths and test-data paths

In [ ]:
def read_image_from_path_as_array(imagePath, width = 600, height = 600, normalization = True): # called inside cross_validation Function
    global Step_no, rI
    if rI == 1:
        print(str(Step_no) + ": read_image_from_path_as_array Function called.")
    Step_no, rI = Step_no + 1, rI + 1
    img = io.imread(imagePath.split('\n')[0]) # print(img) gives ndarray  # print(width, height)
    imageData = transform.resize(img, (width, height, 3))
    if normalization == True:
        imageData = np.transpose(imageData, (2, 0, 1))
        imageData = [preprocessing.scale(imageData[i]) for i in range(3)]
        imageData = np.transpose(imageData,(1, 2, 0)) # imageData = transform.resize(img,(width, height, 3))
    return imageData

In [ ]:
def _convND(ip, rank, channels): # called inside non_local_block Function
    global Step_no
    print(str(Step_no) + ": _convND Function called.")
    Step_no += 1
    assert rank in [3, 4, 5]  # Rank of input must be 3, 4 or 5
    if rank == 3:
        x = Conv1D(channels, 1, padding = 'same', use_bias = False, kernel_initializer = 'he_normal')(ip)
    elif rank == 4:
        x = Conv2D(channels, (1, 1), padding = 'same', use_bias = False, kernel_initializer = 'he_normal')(ip)
    else:
        x = Conv3D(channels, (1, 1, 1), padding = 'same', use_bias = False, kernel_initializer = 'he_normal')(ip)
    return x

In [ ]:
def non_local_block(ip, intermediate_dim = None, compression = 2, mode = 'embedded', add_residual = True): # called inside Network_Config Function
    global Step_no
    print(str(Step_no) + ": non_local_block Function called.")
    Step_no += 1
    channel_dim = 1 if K.image_data_format() == 'channels_first' else -1
    print("K.image_data_format(): "+str(K.image_data_format()))
    ip_shape = K.int_shape(ip)
    if mode not in ['gaussian', 'embedded', 'dot', 'concatenate']:
        raise ValueError('`mode` must be one of `gaussian`, `embedded`, `dot` or `concatenate`')
    compression = 1 if compression is None else compression
    dim1, dim2, dim3 = None, None, None # check rank to calculate the input shape
    if len(ip_shape) == 3:  # temporal / time series data
        rank = 3
        batchsize, dim1, channels = ip_shape
    elif len(ip_shape) == 4:  # spatial / image data
        rank = 4
        if channel_dim == 1:
           batchsize, channels, dim1, dim2 = ip_shape
        else:
           batchsize, dim1, dim2, channels = ip_shape
    elif len(ip_shape) == 5:  # spatio-temporal / Video or Voxel data
        rank = 5
        if channel_dim == 1:
            batchsize, channels, dim1, dim2, dim3 = ip_shape
        else:
            batchsize, dim1, dim2, dim3, channels = ip_shape
    else:
        raise ValueError('Input dimension has to be either 3 (temporal), 4 (spatial) or 5 (spatio-temporal)')
    if intermediate_dim is None:
        intermediate_dim = channels // 2
        if intermediate_dim < 1:
            intermediate_dim = 1
    else:
        intermediate_dim = int(intermediate_dim)
        if intermediate_dim < 1:
            raise ValueError('`intermediate_dim` must be either `None` or positive integer greater than 1.')
    if mode == 'gaussian':  # Gaussian instantiation
        x1 = Reshape((-1, channels))(ip)  # xi
        x2 = Reshape((-1, channels))(ip)  # xj
        f = dot([x1, x2], axes = 2)
        f = Activation('softmax')(f)
    elif mode == 'dot':  # Dot instantiation
        theta = _convND(ip, rank, intermediate_dim) # theta path
        theta = Reshape((-1, intermediate_dim))(theta)
        phi = _convND(ip, rank, intermediate_dim) # phi path
        phi = Reshape((-1, intermediate_dim))(phi)
        f = dot([theta, phi], axes = 2)
        size = K.int_shape(f)
        f = Lambda(lambda z: (1. / float(size[-1])) * z)(f) # scale the values to make it size invariant
    elif mode == 'concatenate':  # Concatenation instantiation
        raise NotImplementedError('Concatenate model has not been implemented yet')
    else:  # Embedded Gaussian instantiation
        # print("ip, rank, intermediate_dim: " + str(ip) + str(rank) + str(intermediate_dim))
        theta = _convND(ip, rank, intermediate_dim) # theta path
        theta = Reshape((-1, intermediate_dim))(theta)
        phi = _convND(ip, rank, intermediate_dim) # phi path
        phi = Reshape((-1, intermediate_dim))(phi)
        if compression > 1: # shielded computation
            phi = MaxPool1D(compression)(phi)
        f = dot([theta, phi], axes = 2)
        f = Activation('softmax')(f)
    g = _convND(ip, rank, intermediate_dim) # g path
    g = Reshape((-1, intermediate_dim))(g)
    if compression > 1 and mode == 'embedded': # shielded computation
        g = MaxPool1D(compression)(g)
    y = dot([f, g], axes=[2, 1]) # compute output path
    if rank == 3: # reshape to input tensor format starts
        y = Reshape((dim1, intermediate_dim))(y)
    elif rank == 4:
        if channel_dim == -1:
            y = Reshape((dim1, dim2, intermediate_dim))(y)
        else:
            y = Reshape((intermediate_dim, dim1, dim2))(y)
    else:
        if channel_dim == -1:
            y = Reshape((dim1, dim2, dim3, intermediate_dim))(y)
        else:
            y = Reshape((intermediate_dim, dim1, dim2, dim3))(y)
    y = _convND(y, rank, channels) # project filters starts
    if add_residual: # residual connection starts
        y = concatenate([ip, y], axis = 3)
    return y

In [ ]:
def squeeze_excitation_layer(x, out_dim, ratio = 4, concate = True): # called inside Network_Config Function # inter-channel weighting.
    global Step_no
    print(str(Step_no) + ": squeeze_excitation_layer Function called.")
    Step_no += 1
    squeeze = GlobalAveragePooling2D()(x)
    excitation = Dense(units=out_dim // ratio)(squeeze)
    excitation = Activation('relu')(excitation)
    excitation = Dense(units=out_dim)(excitation)
    excitation = Activation('sigmoid')(excitation)
    excitation = Reshape((1, 1, out_dim))(excitation)
    scale = multiply([x, excitation])
    if concate:
        scale = concatenate([scale, x], axis=3)
    return scale

In [ ]:
def create_directory(dir_path): # called inside Network_Config Function
    global Step_no
    print(str(Step_no) + ": create_directory Function called.")
    Step_no += 1
    print(dir_path + " is created using create_directory Function." )
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

In [ ]:
def dataAugmentation(input_array, flip_left_right_rate = 0.5, flip_top_bottom_rate = 0.5): # called inside batch_generator Function
    global Step_no, dA
    if dA == 1:
        print(str(Step_no) + ": dataAugmentation Function called.")
    Step_no, dA = Step_no + 1, dA + 1
    if (random.random() < flip_left_right_rate): # Flip left and right
        input_array = np.transpose(input_array, (2, 0, 1)) # w, h, 3 -> 3, w, h
        input_array = [np.flip(input_array[i], 1) for i in range(3)]
        input_array = np.transpose(input_array, (1, 2, 0)) # 3, w, h -> w, h, 3
    if (random.random() < flip_top_bottom_rate): # Flip upside down
        input_array = np.transpose(input_array, (2, 0, 1))  # w, h, 3 -> 3, w, h
        input_array = [np.flip(input_array[i], 0) for i in range(3)]
        input_array = np.transpose(input_array, (1, 2, 0))  # 3, w, h -> w, h, 3
    return np.array(input_array)

In [ ]:
def batch_generator(all_data, all_label, batch_size, shuffle, class_num = 4, train = True): # called inside Network_Config Function
    global Step_no
    print(str(Step_no) + ": batch_generator Function called.")
    Step_no += 1
    assert len(all_data) == len(all_label)
    print("Size of all_data in batch_generator function: "+ str(len(all_data)))
    if shuffle:
        indices = np.arange(len(all_data))
        random.shuffle(indices)
    while True:
        for start_idx in range(0, len(all_data) - batch_size + 1, batch_size):
            data, labels = [], []
            excerpt = indices[start_idx : start_idx + batch_size] if shuffle else slice(start_idx, start_idx + batch_size)
            for di in excerpt:
                tmp_data = all_data[di]
                if train:
                    tmp_data = dataAugmentation(tmp_data)
                data.append(tmp_data)
            for li in excerpt:
                cla, tmp = all_label[li], [0 for x in range(class_num)]
                tmp[cla] = 1
                labels.append(tmp)
            yield np.array(data), np.array(labels)
    print("-------------------------------------------------------------------------dA: " + str(dA))

In [ ]:
def batch_generator_confusion_matrix(all_data, all_label, batch_size, shuffle, class_num = 4): # called inside Network_Config method
    global Step_no
    print(str(Step_no) + ": batch_generator_confusion_matrix Function called.")
    Step_no += 1
    assert len(all_data) == len(all_label)
    if shuffle:
        indices = np.arange(len(all_data))
        random.shuffle(indices)
    for start_idx in range(0, len(all_data) - batch_size + 1, batch_size):
        data, labels = [], []
        excerpt = indices[start_idx : start_idx + batch_size] if shuffle else slice(start_idx, start_idx + batch_size)
        for di in excerpt:
            tmp_data = all_data[di]
            data.append(all_data[di])
        for li in excerpt:
            cla, tmp = all_label[li], [0 for x in range(class_num)]
            tmp[cla] = 1
            labels.append(tmp)
        yield np.array(data), np.array(labels)

In [ ]:
def Network_Config(Test_data, Test_labels, Train_data = None, Train_labels = None, No_of_categories = 0, No_of_epochs = 0, Initial_epoch = 0, Batch_size = 127, Iteration_no = 0): # called inside cross_validation Function
    global Step_no
    print(str(Step_no) + ": Network_Config Function called.")
    Step_no += 1
    adam = Adam(learning_rate = 0.005, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08, decay = 0.0009)
    sgd = SGD(learning_rate = 0.001, momentum = 0.9, decay = 0.0, nesterov = False)
    input_tensor = Input(shape = (224, 224, 3))
    K.clear_session()
    base_model = VGG16(input_tensor = input_tensor, weights = 'imagenet', include_top = False)
    base_output = base_model.output
    x = non_local_block(base_output, intermediate_dim = None, compression = 2, mode = 'embedded', add_residual = False)
    x = BatchNormalization()(x)
    y = squeeze_excitation_layer(base_output, 512, ratio = 4, concate = False)
    y = BatchNormalization()(y)
    x = concatenate([base_output, x], axis = 3)
    x = concatenate([x, y], axis = 3)
    gap = GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    x = concatenate([gap, x])
    x = Dense(512, activation = 'relu')(x)
    x = BatchNormalization()(x)
    x = Dense(512, activation = 'relu')(x)
    x = BatchNormalization()(x)
    x = Dense(512, activation = 'relu')(x)
    x = BatchNormalization()(x)
    predict = Dense(No_of_categories, activation = 'softmax')(x)
    model = Model(inputs = input_tensor, outputs = predict)
    for layer in (base_model.layers):
        layer.trainable = False
    for l in model.layers:
        print(l.name)
    model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = [keras.metrics.categorical_accuracy])
    print("<--------------------------------------------------Model Summary--------------------------------------------------------->")
    model.summary()
    print("------------------------------------------------End of model summary------------------------------------------------------")
    create_directory('./final/')
    weights_file = './final/' + str(Iteration_no) + '-weights.{epoch:02d}-{categorical_accuracy:.4f}-{val_loss:.4f}-{val_categorical_accuracy:.4f}.h5'
    csv_file = './final/record.csv'
    lr_reducer = ReduceLROnPlateau(monitor = 'categorical_accuracy', factor = 0.2, cooldown = 0, patience = 2, min_lr = 0.5e-6)
    early_stopper = EarlyStopping(monitor = 'val_categorical_accuracy', min_delta = 1e-4, patience = 30)
    model_checkpoint = ModelCheckpoint(weights_file, monitor = 'val_categorical_accuracy', save_best_only = True, verbose = 1, save_weights_only = True, mode = 'max')
    tensorboard = TensorBoard(log_dir = './logs/', histogram_freq = 0, write_graph = True, write_images = True, embeddings_freq = 0, embeddings_layer_names = None,
                              embeddings_metadata = None)
    CSV_record = CSVLogger(csv_file, separator = ',', append = True)
    callbacks = [lr_reducer, early_stopper, model_checkpoint, tensorboard, CSV_record]
    gc.disable()
    print("Printing test label: "+ str(Test_labels))
    model.fit_generator(
        generator = batch_generator(np.array(Train_data), np.array(Train_labels), Batch_size, True, No_of_categories, True),
        steps_per_epoch = int(len(Train_labels)/Batch_size)-1,
        max_queue_size = 20,
        initial_epoch = Initial_epoch,
        epochs = No_of_epochs,
        verbose = 1,
        callbacks = callbacks,
        validation_data = batch_generator(np.array(Test_data), np.array(Test_labels), Batch_size, True, No_of_categories, False),
        validation_steps = int( len(Test_labels)/Batch_size )-1 # class_weight=NULL
        )
    print("<--------------------------------------------------Confusion Matrix process starts-------------------------------------------------->")
    Total_predicted_labels, Total_true_labels = [], []
    print("Batch size: " + str(Batch_size))
    for test_data_batch, test_labels_batch in batch_generator_confusion_matrix(np.array(Test_data), np.array(Test_labels), Batch_size, True, No_of_categories):
        print("len(test_data_batch): "+str(len(test_data_batch)))
        y_pred = model.predict(test_data_batch, Batch_size)
        y_true = test_labels_batch
        for y_p in y_pred:
            Total_predicted_labels.append(np.where(y_p == max(y_p))[0][0])
        for y_t in y_true:
            Total_true_labels.append(np.where(y_t == max(y_t))[0][0])
    confusion = confusion_matrix(y_true = Total_true_labels, y_pred = Total_predicted_labels)
    print("Total_predicted_labels: " + str(len(Total_predicted_labels)) )
    print("----------------------------------Printing confusion matrix for the iteration - " + str(Iteration_no) + "---------------------------------")
    print(confusion)
    f = open('confusion_matrix.txt','a+')
    f.write(str(Total_true_labels)+"\n")
    f.write(str(Total_predicted_labels)+"\n")
    f.write(str(confusion)+'\n')
    f.close()
    gc.enable()

In [ ]:
def cross_validation(categorised_data, K, no_of_epochs, no_of_classes, batch_size): # called inside main Function
    global Step_no
    print(str(Step_no) + ": cross_validation Function called.")
    Step_no += 1
    no_of_categories = len(categorised_data) # categorised_data = array of size 4, each index has an array of NE, EP, EH, EA
    for each_category in range(no_of_categories):
        random.shuffle(categorised_data[each_category])
    print("<--------------------------------------------------Iterations initiated-------------------------------------------------->")
    for iteration in range(K):
        print("Iteration no: %d" %iteration)
        train_data_for_present_iteration, test_data_for_present_iteration, train_data_paths, train_data_labels, test_data_paths, test_data_labels = [], [], [], [], [], []
        for category_no in range(no_of_categories):
            train_data_paths_of_each_cat, test_data_paths_of_each_cat = slice_train_test(categorised_data[category_no], iteration, K)
            print("Category-wise train-size: " + str(len(train_data_paths_of_each_cat))+ ", Category-wise test-size: " + str(len(test_data_paths_of_each_cat)))
            for each_path in range(len(train_data_paths_of_each_cat)):
                train_data_paths.append(train_data_paths_of_each_cat[each_path])
                train_data_labels.append(category_no)
            for each_path in range(len(test_data_paths_of_each_cat)):
                test_data_paths.append(test_data_paths_of_each_cat[each_path])
                test_data_labels.append(category_no)
        print("Size of training dataset: " + str(len(train_data_paths)) + ", no. of training labels: " + str(len(train_data_labels)))
        print("Size of testing dataset: " + str(len(test_data_paths)) + ", no. of testing labels: " + str(len(test_data_labels)))
        record = open("record_of_iterations.txt", 'a+')
        record.write("Iteration no: " + str(iteration))
        record.write(str(train_data_paths)+'\n')
        record.write(str(test_data_paths)+'\n')
        record.close()
        for each_path in train_data_paths:
            train_data_for_present_iteration.append(read_image_from_path_as_array(each_path, 224, 224, True))
        for each_path in test_data_paths:
            test_data_for_present_iteration.append(read_image_from_path_as_array(each_path, 224, 224, True))
        Network_Config(Train_data = train_data_for_present_iteration, No_of_categories = no_of_categories, No_of_epochs = no_of_epochs, Train_labels = train_data_labels,
                       Test_data = test_data_for_present_iteration, Test_labels = test_data_labels, Iteration_no = iteration, Initial_epoch = 0, Batch_size = batch_size)


In [ ]:
def main():
    global Step_no
    print(str(Step_no) + ": main Function called.")
    Step_no += 1
    list = creat_list("file_list.txt")
    cross_validation(categorised_data = list, K = 10, no_of_epochs = 12, no_of_classes = 4, batch_size = 32)
main()

1: main Function called.
2: creat_list Function called.
3302 {'NE': 1333, 'EP': 636, 'EH': 798, 'EA': 535}
3: cross_validation Function called.
<--------------------------------------------------Iterations initiated-------------------------------------------------->
Iteration no: 0
4: slice_train_test Function called.
Category-wise train-size: 1200, Category-wise test-size: 133
5: slice_train_test Function called.
Category-wise train-size: 573, Category-wise test-size: 63
6: slice_train_test Function called.
Category-wise train-size: 719, Category-wise test-size: 79
7: slice_train_test Function called.
Category-wise train-size: 482, Category-wise test-size: 53
Size of training dataset: 2974, no. of training labels: 2974
Size of testing dataset: 328, no. of testing labels: 328
8: read_image_from_path_as_array Function called.
3310: Network_Config Function called.
58900480/58889256 [==============================] - 2s 0us/step
3311: non_local_block Function called.
K.image_data_format()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


3318: batch_generator Function called.
Size of all_data in batch_generator function: 2974
3319: dataAugmentation Function called.
Epoch 1/12
91/91 [==============================] - ETA: 0s - loss: 1.4363 - categorical_accuracy: 0.49866263: batch_generator Function called.
Size of all_data in batch_generator function: 328

Epoch 1: val_categorical_accuracy improved from -inf to 0.42361, saving model to ./final/0-weights.01-0.4986-3.0434-0.4236.h5
91/91 [==============================] - 75s 552ms/step - loss: 1.4363 - categorical_accuracy: 0.4986 - val_loss: 3.0434 - val_categorical_accuracy: 0.4236 - lr: 0.0050
Epoch 2/12
91/91 [==============================] - ETA: 0s - loss: 0.9715 - categorical_accuracy: 0.5800
Epoch 2: val_categorical_accuracy improved from 0.42361 to 0.48958, saving model to ./final/0-weights.02-0.5800-1.3530-0.4896.h5
91/91 [==============================] - 31s 340ms/step - loss: 0.9715 - categorical_accuracy: 0.5800 - val_loss: 1.3530 - val_categorical_accura

/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `jupyter nbconvert --to html /content/EC(M)'
